In [18]:
from concorde.problem import Problem
from concorde.concorde import Concorde
import numpy as np
from python_tsp.exact import solve_tsp_branch_and_bound
from python_tsp.heuristics import solve_tsp_lin_kernighan, solve_tsp_record_to_record
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error

In [7]:
X = np.load('X_20x20.npy')
Y = np.load('Y_20x20.npy')

In [8]:
def reformulate_ATSP_as_TSP(x, inf=99, neg_inf=0):
    np.fill_diagonal(x, neg_inf)
    a = np.concatenate((np.full(x.shape, inf), np.transpose(x)), axis=1)
    b = np.concatenate((x, np.full(x.shape, inf)), axis=1)
    return np.concatenate((a, b), axis=0)

def route_ATSP_from_TSP(a, route):
    n = a.shape[0]//2
    route1 = [x for x in route if x < n]
    route2 = [x for x in route[::-1] if x < n]
    dist1 = sum(a[route1[i]+n,route1[(i+1)%n]] for i in range(n))
    dist2 = sum(a[route2[i]+n,route2[(i+1)%n]] for i in range(n))
    if dist2 > dist1:
        return route1, dist1
    else:
        return route2, dist2

In [4]:
a = (reformulate_ATSP_as_TSP(X[-1]) * 10000).astype(int)
problem = Problem.from_matrix(a)
#problem.to_tsp('my_tsp.tsp')

In [5]:
c = Concorde()
solution = c.solve(problem, concorde_exe = '/home/vladimir/src/concorde/TSP/concorde')

In [6]:
%timeit c.solve(problem, concorde_exe = '/home/vladimir/src/concorde/TSP/concorde')

15.7 ms ± 743 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
route_ATSP_from_TSP(a/10000, solution.tour)

([19, 6, 3, 2, 15, 8, 1, 10, 9, 4, 17, 7, 18, 5, 12, 14, 16, 11, 13, 0],
 np.float64(244.58890000000005))

In [7]:
solve_tsp_branch_and_bound(X[-1])

([0, 19, 6, 3, 2, 15, 8, 1, 10, 9, 4, 17, 7, 18, 5, 12, 14, 16, 11, 13],
 np.float64(244.59001031102923))

In [9]:
%timeit solve_tsp_branch_and_bound(X[0])

155 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
route, _ = solve_tsp_lin_kernighan(a)
route_ATSP_from_TSP(a/10000, route)

([0, 19, 9, 12, 15, 18, 11, 13, 4, 8, 1, 10, 16, 14, 17, 7, 5, 6, 3, 2],
 np.float64(278.88660000000004))

In [11]:
%timeit solve_tsp_lin_kernighan(a)

4.39 ms ± 73.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
route, _ = solve_tsp_record_to_record(a)
route_ATSP_from_TSP(a/10000, route)

([0, 16, 14, 7, 18, 5, 19, 8, 6, 3, 4, 1, 10, 9, 12, 2, 15, 17, 11, 13],
 np.float64(247.68540000000002))

In [20]:
%timeit solve_tsp_record_to_record(a)

76 ms ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
border = 60000
X_test = X[border:]
Y_test = Y[border:]

In [16]:
ld = []
ll = []
N = X_test.shape[1]
for i in tqdm(range(X_test.shape[0])):
    a = X_test[i]
    route = Y_test[i]
    distance = sum(a[route[j],route[j+1]] for j in range(N-1))+a[route[-1],route[0]]
    a = (reformulate_ATSP_as_TSP(a) * 10000).astype(int)
    _, ldist = solve_tsp_lin_kernighan(a)
    ld.append(distance)
    ll.append(ldist)

100%|██████████| 3000/3000 [00:13<00:00, 227.48it/s]


In [19]:
Y_true = np.array(ld)
Y_l = np.array(ll)/10000
mean_absolute_percentage_error(Y_true, Y_l)

0.12919041027932046